In [1]:
import torch
from ultralytics import YOLO
import time
import yaml

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# 데이터셋 경로 설정
data_root = 'D:\\Falldown\\Dataset\\Resized_Dataset'
train_root = f'{data_root}\\Train\\images'
val_root = f'{data_root}\\Val\\images'
test_root = f'{data_root}\\Test\\images'

# 클래스 설정
class_names = {0 : 'Human'} # 팀원 yaml에서는 0을 비낙상, 1을 낙상으로 클래스 설정함
num_classes = len(class_names)

# yaml 설정
yaml_info = {
    'path' : 'D:\\Falldown', # YOLOv8 기본 경로
    'names': class_names,
    'nc': num_classes,
    'train': train_root,
    'val': val_root,
    'test': test_root
}

# YAML 파일 저장 경로
yaml_file_path = 'D:\\Falldown\\yaml_info_yolov8s.yaml'

# YAML 파일 생성
with open(yaml_file_path, 'w') as f:
    yaml.dump(yaml_info, f)

print(f'yaml 파일이 생성되었습니다: {yaml_file_path}')

yaml 파일이 생성되었습니다: D:\Falldown\yaml_info_yolov8s.yaml


In [4]:
model = YOLO('yolov8s.pt')

In [6]:
try:
    # 학습 시작 시간 기록
    start_time = time.time()

    # 학습 실행
    result = model.train(
        data='D:\\Falldown\\yaml_info_yolov8s.yaml',
        epochs=50,
        batch=16,
        imgsz=640,
        device=device,
        workers=12,
        amp=True,
        patience=30,
        name='human_fall_s'
    )

    # 학습 종료 시간 기록
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"실행 시간: {execution_time:.4f} 초")  # 약 20시간 소요

except KeyboardInterrupt:
    print("KeyboardInterrupt: 사용자가 학습을 중단하였습니다.")

engine\trainer: task=detect, mode=train, model=D:\Falldown\runs\detect\human_fall_s30\weights\best.pt, data=D:\Falldown\yaml_info_yolov8s.yaml, epochs=50, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=12, project=None, name=human_fall_s, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None,

RuntimeError: Dataset 'D://Falldown/yaml_info_yolov8s.yaml' error  
Dataset 'D://Falldown/yaml_info_yolov8s.yaml' images not found , missing path 'D:\Falldown\Dataset\Resized_Dataset\Val\images'
Note dataset download directory is 'D:\Falldown\datasets'. You can update this in 'C:\Users\CHO\AppData\Roaming\Ultralytics\settings.json'

In [5]:
# 검증 시작 시간 기록
start_time = time.time()

# YOLO 모델 로드 및 검증 실행
model = YOLO('D:\\Falldown\\runs\\detect\\human_fall_s30\\weights\\best.pt')
val_results = model.val(
    data='D:\\Falldown\\yaml_info_yolov8s.yaml',
    imgsz=640,
    batch=32,
    device=device
)

# 평가 종료 시간 기록
end_time = time.time()
execution_time = end_time - start_time

# 결과 출력
print(f"Validation Results:")
print(f"mAP50: {results.box.map50:.4f}")
print(f"mAP50-95: {results.box.map:.4f}")
print(f"실행 시간: {execution_time:.4f} 초")

Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


FileNotFoundError: 
Dataset 'D://Falldown/yaml_info_yolov8s.yaml' images not found ⚠️, missing path 'D:\Falldown\Dataset\Resized_Dataset\Val\images'
Note dataset download directory is 'D:\Falldown\datasets'. You can update this in 'C:\Users\CHO\AppData\Roaming\Ultralytics\settings.json'

In [ ]:
# 테스트 시작 시간 기록
start_time = time.time()

# 테스트 실행
test_results = model.val(
    data='D:\\Falldown\\yaml_info_yolov8s.yaml',
    imgsz=640,
    batch=32,
    device=device,
    split="test"  # Test 데이터셋으로 평가
)

# 테스트 종료 시간 기록
end_time = time.time()
execution_time = end_time - start_time

# 테스트 결과 출력
print(f"Test Results:")
print(f"mAP50: {test_results.box.map50:.4f}")
print(f"mAP50-95: {test_results.box.map:.4f}")
print(f"실행 시간: {execution_time:.4f} 초")